### 1. RunnableSequence

executes in swquesnce. output of the first sequence will pass to the input of next sequence and so on

In [1]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence

from dotenv import load_dotenv
load_dotenv() 

True

In [2]:
model = GoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17")
parser = StrOutputParser()

prompt = PromptTemplate(template= "write a riddle about {topic}",
                        input_variables=['topic'])

In [3]:
runnable = RunnableSequence(prompt, model, parser)
runnable.invoke({'topic':'python'})

"I speak a language, clear and neat,\nWhere spaces make my structure complete.\nI wear many hats, from web to AI,\nAnd help automate tasks, reaching high.\n\nThough named for a creature that slithers on land,\nI'm a tool for developers, close at hand.\nI'm often the first for those who begin,\nWith a rich set of tools, I help you win.\n\nWhat am I?\n\\\n... Python"

In [4]:
prompt2 = PromptTemplate(template="can you solve the following riddle. \n {riddle}",
                         input_variables=['riddle'])

In [5]:
runnable = RunnableSequence(prompt, model, parser, prompt2, model, parser)
runnable.invoke({'topic':'math'})

'You are absolutely right!\n\nThe answer to the riddle is indeed **Math**.\n\nAll the clues point to it: speaking in numbers, offering problems with multiple solution paths, dealing with patterns and logic, being used in engineering and astronomy, and existing as an abstract concept.\n\nWell done!'

### 2. RunnableParallel

to execute each runnables parallely.

each runnable will get same input,
returns a dict of outputs

In [6]:
from langchain.schema.runnable import RunnableParallel

prompt1 = PromptTemplate(
    template="generate a linkedin post about {topic}",
    input_variables=['topic']
)
prompt2 = PromptTemplate(
    template="generate a twitter post about {topic}",
    input_variables=['topic']
)

runnable = RunnableParallel(
    {
        "linkedin_post": RunnableSequence(prompt1, model, parser),
        "tweet": RunnableSequence(prompt2, model, parser)
        }
)
runnable.invoke({'topic':"AI"})

{'linkedin_post': "Okay, here are a few options for a LinkedIn post about AI, ranging in tone and focus. Choose the one that best fits your perspective!\n\n**Option 1 (Focus: Impact & Opportunity)**\n\nSubject: The AI Wave: Navigating the Present and Shaping the Future\n\nAI isn't just the future; it's the present. And it's accelerating faster than many predicted.\n\nFrom automating routine tasks and boosting productivity to unlocking new insights and driving innovation, Artificial Intelligence is fundamentally reshaping industries and roles.\n\nFor professionals, understanding and engaging with AI isn't just about staying current – it's about identifying new opportunities, enhancing capabilities, and preparing for the evolving landscape of work.\n\nWhile it brings crucial conversations around ethics, job evolution, and responsible development, the potential for positive impact across business, healthcare, education, and beyond is immense.\n\nHow is AI impacting *your* role or industry

### 3. RunnablePassthrough

it return the input as it is as output

In [7]:
from langchain.schema.runnable import RunnablePassthrough

RunnablePassthrough().invoke(2)

2

In [8]:
prompt1 = PromptTemplate(template="tell me a joke about {topic}",
                         input_variables=['topic'])

prompt2 = PromptTemplate(template="explain me the joke below \n {topic}",
                         input_variables=['topic'])

In [9]:
runnable_sequence = RunnableSequence(prompt, model, parser)

runnable_parallel = RunnableParallel({'the_joke': RunnablePassthrough(),
                                      'explanation': RunnableSequence(prompt, model, parser)})

RunnableSequence(runnable_sequence, runnable_parallel).invoke({"topic":'AI'})

{'the_joke': "Okay, here's a riddle about AI:\n\nI have no body, yet I learn and grow.\nI have no voice, but I can speak and write.\nI see patterns in data, though I have no sight.\nI am made of code, living in the digital light.\nI get smarter the more I'm taught.\n\nWhat am I?\n\\\n... Artificial Intelligence (AI)",
 'explanation': 'Okay, here is another riddle about AI:\n\nI have no body, but I can learn and decide.\nI process information, with nowhere to hide.\nI speak without a mouth, and write without a hand.\nI live in the cloud, across the land.\nI find patterns in data, vast and grand.\n\nWhat am I?\n\\\n... Artificial Intelligence (AI)'}

### 4. RunnableLambda

it allows custom python script into runnables

In [68]:
from langchain.schema.runnable import RunnableLambda
import re


def extract_python_code_and_execute(texts):
    python_codes = re.findall(r'```python(.*)```',texts, re.DOTALL)
    
    if python_codes:
        combined_python_code = "\n".join(python_codes)
        
        exec(combined_python_code)
        return "Code executed successfully"
    else:
        return "No Python code found"


prompt1 = PromptTemplate(template="""Give me a full python code to {question}""",
                         input_variables=['question'])

runnable1 = RunnableParallel({
    'the_code': RunnablePassthrough(),
    'result' : RunnableLambda(extract_python_code_and_execute)
})

runnable2 = RunnableSequence(prompt1,model,parser,runnable1)

runnable2.invoke({"question":'print prime numbers between 1 till 100'})

Prime numbers between 1 and 100 are:
2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97


{'the_code': 'Okay, here is the full Python code to find and print prime numbers between 1 and 100.\n\nThis code includes a helper function `is_prime()` to check if a single number is prime, and then it loops through the range 1 to 100, calling the helper function for each number.\n\n```python\n# Function to check if a number is prime\ndef is_prime(num):\n    """\n    Checks if a number is prime.\n\n    Args:\n        num: The integer to check.\n\n    Returns:\n        True if the number is prime, False otherwise.\n    """\n    # Prime numbers must be greater than 1\n    if num <= 1:\n        return False\n    # 2 is the only even prime number\n    if num == 2:\n        return True\n    # All other even numbers are not prime\n    if num % 2 == 0:\n        return False\n    # Check for odd divisors from 3 up to the square root of the number\n    # We only need to check up to the square root because if a number has\n    # a divisor greater than its square root, it must also have one\n   

### RunnableBranch

conditional logic

In [69]:
from langchain.schema.runnable import RunnableBranch

In [70]:
prompt1 = PromptTemplate(template="Give me report about {topic}",
                         input_variables=['topic'])

prompt2 = PromptTemplate(template="can you summarize below topic in 4 sentences. \n {topic}", 
                         input_variables=['topic'])
cond_runnable = RunnableBranch(
    ##  (condition1, runnable)
    (lambda x: len(x.split())>150, RunnableSequence(prompt2, model, parser) ),
    RunnablePassthrough()
    )


RunnableSequence(prompt1, model, parser, cond_runnable).invoke({"topic":"AI"})

'Artificial Intelligence (AI) is a field dedicated to creating systems that perform tasks typically requiring human intelligence, such as learning, reasoning, and perception. It encompasses various types like Narrow AI and leverages key technologies including Machine Learning, Deep Learning, and Natural Language Processing. AI is being applied across almost all sectors, driving innovation and efficiency in areas from healthcare to finance. However, its development also poses significant ethical, social, and safety challenges that require careful consideration alongside its rapid evolution and future potential.'

## LCEL 

RunnableSequence(r1, r2, r3)  ---->    r1 | r2 | r3